In [0]:
##Connections keys

In [0]:
dbutils.fs.ls("abfss://raw@storage2de.dfs.core.windows.net/")
dbutils.fs.ls("abfss://backups@storage2de.dfs.core.windows.net/")
dbutils.fs.ls("abfss://logs@storage2de.dfs.core.windows.net/")

In [0]:
dbutils.fs.ls("abfss://raw@storage2de.dfs.core.windows.net/")

In [0]:
#Assigning the schema as needed
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

hired_employees_schema = StructType([
    StructField("id", IntegerType(), True),  # id del empleado
    StructField("name", StringType(), True),  # nombre y apellido del empleado
    StructField("datetime", StringType(), True),  # fecha y hora de contratación
    StructField("department_id", IntegerType(), True),  # id del departamento
    StructField("job_id", IntegerType(), True)  # id del trabajo
])

departments_schema = StructType([
    StructField("id", IntegerType(), True),  # id del empleado
    StructField("department", StringType(), True),  # id del departamento
])


jobs_schema = StructType([
    StructField("id", IntegerType(), True),  # id del empleado
    StructField("job", StringType(), True)  # id del trabajo
])


In [0]:
hired_employees_df = spark.read.csv("abfss://raw@storage2de.dfs.core.windows.net/hired_employees.csv", header=False, schema=hired_employees_schema)
departments_df = spark.read.csv("abfss://raw@storage2de.dfs.core.windows.net/departments.csv", header=False, schema=departments_schema)
jobs_df = spark.read.csv("abfss://raw@storage2de.dfs.core.windows.net/jobs.csv", header=False,schema=jobs_schema)

In [0]:
display(departments_df)

In [0]:
from pyspark.sql.functions import col

invalid_hired = hired_employees_df.filter(
    col("id").isNull() | col("name").isNull() | col("datetime").isNull() |
    col("department_id").isNull() | col("job_id").isNull()
)
invalid_hired.write.mode("append").csv("abfss://logs@storage2de.dfs.core.windows.net/")
valid_hired = hired_employees_df.na.drop()

In [0]:
invalid_departments = departments_df.filter(
    col("id").isNull() | col("department").isNull()
)
invalid_departments.write.mode("append").csv("abfss://logs@storage2de.dfs.core.windows.net/")
valid_departments = departments_df.na.drop()

In [0]:
invalid_jobs = jobs_df.filter(
    col("id").isNull() | col("job").isNull()
)
invalid_jobs.write.mode("append").csv("abfss://logs@storage2de.dfs.core.windows.net/")
valid_jobs = jobs_df.na.drop()

In [0]:
# Conexión JDBC
jdbc_url = "jdbc:sqlserver://db-sql-db.database.windows.net:1433;databaseName=db-de-v2;user=adminsql;password=password123.;encrypt=true;trustServerCertificate=false"

# Configura las propiedades de conexión
connection_properties = {
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


valid_departments.write.jdbc(url=jdbc_url, table="departments", mode="append", properties=connection_properties)

# Cargar trabajos (jobs)
valid_jobs.write.jdbc(url=jdbc_url, table="jobs", mode="append", properties=connection_properties)

# Ahora sí cargar empleados
valid_hired.write.jdbc(url=jdbc_url, table="hired_employees", mode="append", properties=connection_properties)

In [0]:
jdbc_url = "jdbc:sqlserver://sv-de-globant-challenge.database.windows.net:1433;databaseName=db-de-globant;user=joshilope1609;password=<tu_contraseña>;encrypt=true;trustServerCertificate=false"

connection_properties = {
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
jdbc_url = "jdbc:sqlserver://sv-de-globant-challenge.database.windows.net:1433;database=db-de-globant;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;Authentication=ActiveDirectoryIntegrated"

connection_properties = {
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

valid_hired.write.jdbc(url=jdbc_url, table="hired_employees", mode="append", properties=connection_properties)